# Customers of E-Commerce Data

## import Libraries

In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import warnings
warnings.filterwarnings('ignore')


plt.style.use('fivethirtyeight')
sns.set_style("whitegrid")
pd.set_option('display.max_columns', None)

In [2]:
# reading data
df = pd.read_csv('./E-commerce Customer Behavior - Sheet1.csv')

In [3]:
df

,Customer ID,Gender,Age,City,Membership Type,Total Spend,Items Purchased,Average Rating,Discount Applied,Days Since Last Purchase,Satisfaction Level
0,101,Female,29,New York,Gold,1120.20,14,4.6,True,25,Satisfied
1,102,Male,34,Los Angeles,Silver,780.50,11,4.1,False,18,Neutral
2,103,Female,43,Chicago,Bronze,510.75,9,3.4,True,42,Unsatisfied
3,104,Male,30,San Francisco,Gold,1480.30,19,4.7,False,12,Satisfied
4,105,Male,27,Miami,Silver,720.40,13,4.0,True,55,Unsatisfied
...,...,...,...,...,...,...,...,...,...,...,...
345,446,Male,32,Miami,Silver,660.30,10,3.8,True,42,Unsatisfied
346,447,Female,36,Houston,Bronze,470.50,8,3.0,False,27,Neutral
347,448,Female,30,New York,Gold,1190.80,16,4.5,True,28,Satisfied
348,449,Male,34,Los Angeles,Silver,780.20,11,4.2,False,21,Neutral


## EDA

`Data Types and Non-Null Count`

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer ID               350 non-null    int64  
 1   Gender                    350 non-null    object 
 2   Age                       350 non-null    int64  
 3   City                      350 non-null    object 
 4   Membership Type           350 non-null    object 
 5   Total Spend               350 non-null    float64
 6   Items Purchased           350 non-null    int64  
 7   Average Rating            350 non-null    float64
 8   Discount Applied          350 non-null    bool   
 9   Days Since Last Purchase  350 non-null    int64  
 10  Satisfaction Level        348 non-null    object 
dtypes: bool(1), float64(2), int64(4), object(4)
memory usage: 27.8+ KB


In [5]:
print("\nDescriptive Statistics:")
display(df.describe().round(2))


Descriptive Statistics:


,Customer ID,Age,Total Spend,Items Purchased,Average Rating,Days Since Last Purchase
count,350.00,350.00,350.00,350.00,350.00,350.00
mean,275.50,33.60,845.38,12.60,4.02,26.59
std,101.18,4.87,362.06,4.16,0.58,13.44
min,101.00,26.00,410.80,7.00,3.00,9.00
25%,188.25,30.00,502.00,9.00,3.50,15.00
50%,275.50,32.50,775.20,12.00,4.10,23.00
75%,362.75,37.00,1160.60,15.00,4.50,38.00
max,450.00,43.00,1520.10,21.00,4.90,63.00


`show the last 5 rows of the dataset`

In [6]:
df.tail()

,Customer ID,Gender,Age,City,Membership Type,Total Spend,Items Purchased,Average Rating,Discount Applied,Days Since Last Purchase,Satisfaction Level
345,446,Male,32,Miami,Silver,660.30,10,3.8,True,42,Unsatisfied
346,447,Female,36,Houston,Bronze,470.50,8,3.0,False,27,Neutral
347,448,Female,30,New York,Gold,1190.80,16,4.5,True,28,Satisfied
348,449,Male,34,Los Angeles,Silver,780.20,11,4.2,False,21,Neutral
349,450,Female,43,Chicago,Bronze,515.75,10,3.3,True,49,Unsatisfied


`checking if any column has null values`

In [7]:
df.isnull().sum()

,0
Customer ID,0
Gender,0
Age,0
City,0
Membership Type,0
Total Spend,0
Items Purchased,0
Average Rating,0
Discount Applied,0
Days Since Last Purchase,0


In [8]:
df.dropna(inplace=True)

In [9]:
print(f"data shape after removing missing values: {df.shape}")

data shape after removing missing values: (348, 11)


In [10]:
# checking the duplication in the dt
df.duplicated().sum()

np.int64(0)

`the Satisfaction Level have two missing values`

### checking for dt outliers in numerical columns

In [11]:
# using IQR
def detect_outliers(df,col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3-Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    return outliers.shape[0] , lower_bound, upper_bound

In [12]:
numeric_cols = ['Age', 'Total Spend', 'Items Purchased', 'Average Rating', 'Days Since Last Purchase']
outlier_summary = []

In [13]:
for col in numeric_cols:
    num_outliers, lower_bound, upper_bound = detect_outliers(df, col)
    outlier_summary.append({
        'Column': col,
        'Outliers Count': num_outliers,
        'Lower Bound': round(lower_bound, 2),
        'Upper Bound': round(upper_bound, 2),
        '% of Data': round((num_outliers / len(df)) * 100, 2)
    })
outlier_df = pd.DataFrame(outlier_summary)

In [14]:
outlier_df

,Column,Outliers Count,Lower Bound,Upper Bound,% of Data
0,Age,0,19.50,47.50,0.0
1,Total Spend,0,-476.52,2142.88,0.0
2,Items Purchased,0,0.00,24.00,0.0
3,Average Rating,0,2.00,6.00,0.0
4,Days Since Last Purchase,0,-19.50,72.50,0.0


#### Age Distribution

In [15]:
fig = px.histogram(df, x='Age', color='Gender',
                   marginal='box',
                   title='Age Distribution by Gender',
                   color_discrete_map={'Male': '#636EFA', 'Female': '#EF553B'},
                   opacity=0.7)
fig.update_layout(bargap=0.1,
                 legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1))
fig.show()

#### Gender Distribution

In [16]:
gender_counts = df['Gender'].value_counts().reset_index()
gender_counts

,Gender,count
0,Male,175
1,Female,173


In [17]:
fig = px.pie(gender_counts, values='count', names='Gender',
             title='Gender Distribution',
             color_discrete_map={'Male': '#636EFA', 'Female': '#EF553B'},
             hole=0.4)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=-0.1, xanchor="center", x=0.5))
fig.show()

#### City Distribution

In [18]:
city_counts = df['City'].value_counts().reset_index()
city_counts

,City,count
0,New York,59
1,Los Angeles,59
2,Chicago,58
3,San Francisco,58
4,Miami,58
5,Houston,56


In [19]:
city_counts['Percentage'] = (city_counts['count'] / city_counts['count'].sum() * 100).round(1)

In [20]:
fig = px.bar(city_counts, x='City', y='count', text='Percentage',
             title='Customer Distribution by City',
             color='count', color_continuous_scale='Viridis')
fig.update_traces(texttemplate='%{text}%', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',
                 xaxis_title='City', yaxis_title='Number of Customers')
fig.show()

#### City & Age

In [21]:
fig = px.violin(df, x='City', y='Age', color='Gender', box=True, points="all",
                title='Age Distribution Across Cities by Gender',
                color_discrete_map={'Male': '#636EFA', 'Female': '#EF553B'})
fig.update_layout(xaxis_title='City', yaxis_title='Age')
fig.show()

#### Membership Type Distribution

In [22]:
membership_counts = df['Membership Type'].value_counts().reset_index()
membership_counts

,Membership Type,count
0,Gold,117
1,Silver,117
2,Bronze,114


In [23]:
fig = px.pie(membership_counts, values='count', names='Membership Type',
             title='Membership Type Distribution',
             color_discrete_sequence=px.colors.sequential.Plasma_r,
             hole=0.4)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=-0.1, xanchor="center", x=0.5))
fig.show()

##### Total spend by Membership Type

In [24]:
fig = px.box(df, x='Membership Type', y='Total Spend', color='Membership Type',
             title='Total Spend Distribution by Membership Type',
             color_discrete_sequence=px.colors.sequential.Plasma_r)
fig.update_layout(showlegend=False, xaxis_title='Membership Type', yaxis_title='Total Spend ($)')
fig.show()

##### Correlation between Total Spend and Items Purchased with regression

In [25]:
fig = px.scatter(df, x='Items Purchased', y='Total Spend',
                color='Membership Type', size='Average Rating',
                hover_data=['Customer ID', 'Age', 'Gender', 'City'],
                title='Relationship between Items Purchased and Total Spend',
                trendline='ols',
                color_discrete_sequence=px.colors.sequential.Plasma_r)
fig.update_layout(xaxis_title='Items Purchased', yaxis_title='Total Spend ($)')
fig.show()

##### Calculate correlation coefficient

In [28]:
corr_coef = df['Total Spend'].corr(df['Items Purchased'])
print(corr_coef)

0.972276360244477


###### Average Spending by gender and membership

In [30]:
gender_membership_spend = df.groupby(['Gender', 'Membership Type'])['Total Spend'].mean().reset_index()
gender_membership_spend

,Gender,Membership Type,Total Spend
0,Female,Bronze,474.223684
1,Female,Gold,1164.250000
2,Female,Silver,730.400000
3,Male,Gold,1455.549153
4,Male,Silver,748.587931


In [31]:
fig = px.bar(gender_membership_spend, x='Membership Type', y='Total Spend', color='Gender',
             barmode='group', title='Average Spending by Gender and Membership Type',
             color_discrete_map={'Male': '#636EFA', 'Female': '#EF553B'})
fig.update_layout(xaxis_title='Membership Type', yaxis_title='Average Total Spend ($)')
fig.show()

#### Satisfaction level Distribution

In [32]:
satisfaction_counts = df['Satisfaction Level'].value_counts().reset_index()
satisfaction_counts

,Satisfaction Level,count
0,Satisfied,125
1,Unsatisfied,116
2,Neutral,107


In [34]:
satisfaction_colors = {'Satisfied': '#3D9970', 'Neutral': '#FFD700', 'Unsatisfied': '#FF4136'}
fig = px.bar(satisfaction_counts, x='Satisfaction Level', y='count', color='Satisfaction Level',
             title='Distribution of Customer Satisfaction Levels',
             color_discrete_map=satisfaction_colors)
fig.update_layout(showlegend=False, xaxis_title='Satisfaction Level', yaxis_title='Number of Customers')
fig.show()

##### Average Rating by satisfaction level

In [36]:
avg_rating_by_satisfaction = df.groupby('Satisfaction Level')['Average Rating'].mean().round(2).reset_index()
avg_rating_by_satisfaction

,Satisfaction Level,Average Rating
0,Neutral,3.66
1,Satisfied,4.65
2,Unsatisfied,3.69


In [37]:
fig = px.bar(avg_rating_by_satisfaction, x='Satisfaction Level', y='Average Rating',
             color='Satisfaction Level', title='Average Rating by Satisfaction Level',
             color_discrete_map=satisfaction_colors)
fig.update_layout(showlegend=False, xaxis_title='Satisfaction Level',
                 yaxis_title='Average Rating', yaxis=dict(range=[0, 5]))
fig.show()

##### Rating distribution by satisfaction level

In [38]:
fig = px.histogram(df, x='Average Rating', color='Satisfaction Level',
                   marginal='box', nbins=20,
                   title='Rating Distribution by Satisfaction Level',
                   color_discrete_map=satisfaction_colors)
fig.update_layout(bargap=0.1, xaxis_title='Average Rating', yaxis_title='Count')
fig.show()

##### Satisfaction level by Membership type

In [42]:
satisfaction_by_membership = pd.crosstab(df['Membership Type'], df['Satisfaction Level'])
satisfaction_by_membership_pct = satisfaction_by_membership.div(satisfaction_by_membership.sum(axis=1), axis=0) * 100

In [43]:
satisfaction_by_membership_pct

Satisfaction Level,Neutral,Satisfied,Unsatisfied
Membership Type,,,
Bronze,49.122807,0.000000,50.877193
Gold,0.000000,100.000000,0.000000
Silver,43.589744,6.837607,49.572650


In [44]:
fig = px.bar(satisfaction_by_membership_pct.reset_index().melt(id_vars='Membership Type',
                                                          value_name='Percentage',
                                                          var_name='Satisfaction Level'),
            x='Membership Type', y='Percentage', color='Satisfaction Level',
            title='Satisfaction Level Distribution by Membership Type (%)',
            color_discrete_map=satisfaction_colors)
fig.update_layout(xaxis_title='Membership Type', yaxis_title='Percentage (%)')
fig.show()

#### Purchase Analysis

##### Days Scince last purchase distribution

In [45]:
fig = px.histogram(df, x='Days Since Last Purchase', nbins=30,
                  title='Distribution of Days Since Last Purchase',
                  color_discrete_sequence=['#19A0AA'])
fig.update_layout(xaxis_title='Days Since Last Purchase', yaxis_title='Count')
fig.show()

##### Relation between Days since last purchase by satisfaction level

In [46]:
fig = px.box(df, x='Satisfaction Level', y='Days Since Last Purchase', color='Satisfaction Level',
            title='Days Since Last Purchase by Satisfaction Level',
            color_discrete_map=satisfaction_colors)
fig.update_layout(showlegend=False, xaxis_title='Satisfaction Level',
                 yaxis_title='Days Since Last Purchase')
fig.show()

##### Comparing between Purchase recency and total spend

In [47]:
fig = px.scatter(df, x='Days Since Last Purchase', y='Total Spend',
                color='Satisfaction Level', size='Items Purchased',
                hover_data=['Customer ID', 'Gender', 'Membership Type'],
                title='Relationship between Purchase Recency and Total Spend',
                color_discrete_map=satisfaction_colors)
fig.update_layout(xaxis_title='Days Since Last Purchase', yaxis_title='Total Spend ($)')
fig.show()

#### Discount Analysis

In [48]:
discount_counts = df['Discount Applied'].value_counts().reset_index()
discount_counts

,Discount Applied,count
0,True,175
1,False,173


In [50]:
fig = px.pie(discount_counts, values='count', names='Discount Applied',
             title='Discount Application Distribution',
             color_discrete_sequence=['#1E88E5', '#FFC107'],
             hole=0.4)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=-0.1, xanchor="center", x=0.5))
fig.show()

##### Total spend by discount application

In [51]:
fig = px.box(df, x='Discount Applied', y='Total Spend', color='Discount Applied',
            title='Total Spend Distribution by Discount Application',
            color_discrete_sequence=['#1E88E5', '#FFC107'])
fig.update_layout(showlegend=False, xaxis_title='Discount Applied', yaxis_title='Total Spend ($)')
fig.show()

##### Discount application by membership type

In [52]:
discount_by_membership = pd.crosstab(df['Membership Type'], df['Discount Applied'])
discount_by_membership_pct = discount_by_membership.div(discount_by_membership.sum(axis=1), axis=0) * 100

In [53]:
discount_by_membership_pct

Discount Applied,False,True
Membership Type,,
Bronze,49.122807,50.877193
Gold,49.572650,50.427350
Silver,50.427350,49.572650


In [54]:
fig = px.bar(discount_by_membership_pct.reset_index().melt(id_vars='Membership Type',
                                                     value_name='Percentage',
                                                     var_name='Discount Applied'),
           x='Membership Type', y='Percentage', color='Discount Applied',
           title='Discount Application by Membership Type (%)',
           color_discrete_sequence=['#1E88E5', '#FFC107'])
fig.update_layout(xaxis_title='Membership Type', yaxis_title='Percentage (%)')
fig.show()

#### Customer Segmentation
`using K-Means Clustering`

In [55]:
cluster_features = ['Age', 'Total Spend', 'Items Purchased', 'Average Rating', 'Days Since Last Purchase']
X = df[cluster_features].copy()

In [56]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [57]:
# found optimal num of clusters using Elbow Method
inertia = []
k_range = range(1,11)
for k in k_range:
  kmeans = KMeans(n_clusters= k, random_state=42)
  kmeans.fit(X_scaled)
  inertia.append(kmeans.inertia_)

In [58]:
fig = px.line(x=list(k_range), y=inertia, markers=True,
             title='Elbow Method for Optimal k',
             labels={'x': 'Number of Clusters (k)', 'y': 'Inertia'})
fig.show()

In [59]:
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
df['Cluster'] = kmeans.fit_predict(X_scaled)

In [61]:
df['Cluster'].unique()

array([1, 3, 0, 2], dtype=int32)

#### Cluster Analysis

In [63]:
cluster_analysis = df.groupby('Cluster').agg({
    'Age': 'mean',
    'Total Spend': 'mean',
    'Items Purchased': 'mean',
    'Average Rating': 'mean',
    'Days Since Last Purchase': 'mean',
    'Customer ID': 'count'
}).rename(columns={'Customer ID': 'Count'}).reset_index()
cluster_analysis.round(2)

,Cluster,Age,Total Spend,Items Purchased,Average Rating,Days Since Last Purchase,Count
0,0,39.40,474.22,8.52,3.33,31.78,114
1,1,29.92,1311.14,17.62,4.68,17.94,117
2,2,27.46,698.13,12.41,3.99,51.49,39
3,3,33.60,773.59,11.28,4.08,19.64,78
